In [11]:
!pip install langchain langchain-community langchain-anthropic faiss-cpu sentence-transformers

  Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.6.0 requires tokenizers<0.20.0,>=0.19.1, but you have tokenizers 0.21.2 which is incompatible.


In [12]:
!pip install pymupdf

In [14]:
# .env 파일 로드
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경변수 로드

# 이제 로더 실행
from langchain_upstage import UpstageDocumentParseLoader

file_path = "./test_modified.pdf"

loader = UpstageDocumentParseLoader(
    file_path,
    split="page",  # 페이지별로 분할
    output_format="text",  # 텍스트 형태로 출력
    ocr="auto",  # 자동 OCR 사용
    coordinates=True,  # 좌표 정보 포함
)


docs = loader.load()

In [41]:
print(docs[21].page_content)

3 전형별, 모집단위별 모집인원 1 정원내 계열 대학 모집단위 수시모집 정시모집
 고교 추천 전형 ACE 전형 첨단 융합 인재 전형 고른 기회1 전형 고른 기회2 전형 논술 우수자 전형 체육 우수자 (축구) 전형 나군 다군
 일반 전형1 일반 전형2 일반 전형3 일반 전형4 (교차)
 자연 인문 공과대학 다산학부 기계공학과 23 자유전공학부(인문) 365 42 0 6 5 45 5 0 0 0 33 15 0
 산업공학과 20 23 0 4 2 0 0 0 0 20 0
 화학공학과 5 17 0 1 1 5 0 0 0 7 0
 첨단신소재공학과 15 0 36 2 2 8 0 0 0 22 0
 응용화학과 6 0 17 1 1 0 0 0 0 146 7 0
 환경안전공학과 5 13 0 2 0 0 0 0 0 11 0
 건설시스템공학과 7 13 0 2 91 1 0 0 0 0 7 0
 교통시스템공학과 7 13 0 약학과 3 0 0 0 0 0 7 0
 건축학과 10 23 0 3 2 11 0 0 0 10 0
 융합시스템공학과 0 0 0 0 0 0 0 0 0 0 0
 첨단ICT 융합대학 전자공학과 30 32 0 11 5 37 합계 0 0 0 32 0
 지능형반도체공학과 5 0 자유전공학부(자연) 10 2 1 5 0 0 0 17 0
 미래모빌리티공학과 22 0 42 7 5 14 0 0 0 47 0 0
 소프트웨어 소프트웨어학과 11 16 30 4 2 10 0 0 0 0 0 11 0 0
 사이버보안학과 8 15 0 2 0 0 0 20 0 0 0 0 7 0 의과대학 0
 융합대학 디지털미디어학과 의학과 18 15 42 40 0 0 5 0 2 0 0 0 45 0 0 10 0 0
 국방디지털융합학과 간호학과 0 7 0 25 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 22 0 12 0
 자연과학 대학 간호대학 약학대학 수학과 3 10 0 1 0 0 16 27 0 0 10 0 3 0 0
 프런티어과학학부 18 0 50 0 6 5 0 5 0 0 0 0 0 30 0 0
 인문 자연 자연 경영대학 첨

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#  문서 자르기 (AI가 읽기 쉬운 크기로 쪼개기)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
docs_spliiter = splitter.split_documents(docs)

In [ ]:
len(docs_spliiter)

52

In [ ]:
!pip install -U langchain-huggingface


In [71]:
from langchain_huggingface import HuggingFaceEmbeddings


model_name = "intfloat/multilingual-e5-large-instruct"

hf_embddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device" : "mps"},
    encode_kwargs={"normalize_embeddings" : True},
)

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = HuggingFaceEmbeddings(model="intfloat/multilingual-e5-large-instruct")

dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)



1024


In [ ]:
from langchain_community.vectorstores import FAISS
import faiss
db = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [84]:
# split_doc1 → docs 또는 올바른 변수명 사용
db = FAISS.from_documents(documents=docs, embedding=hf_embddings)

In [85]:
db.index_to_docstore_id

{0: '7ee53fe1-b0b3-4ce4-80c7-b7c51aab7820',
 1: '1ebde446-3fdd-4691-b1fe-c28f173e13f9',
 2: '939816df-792c-40a4-87af-5aa716615b0e',
 3: '7061e3d7-d1e0-4e8e-b4dd-6db4965f246c',
 4: '6285a762-f172-4a1b-b386-0128febb7169',
 5: 'b8476143-b83d-4c99-bee1-b155227714bb',
 6: '8c8f3ebe-64e8-4972-8fed-52def7b29961',
 7: 'a74b3f77-c0a5-4c66-8a59-4193f74efe20',
 8: '798fb1a2-7f41-41ec-b690-051d6eca2c34',
 9: 'fcaeb429-b884-43cb-b92a-86e62932a6aa',
 10: '396698f2-35d3-47fe-885d-766c10a7c794',
 11: '62067638-a419-4edc-86ec-63413cbafed5',
 12: '49918314-989c-4818-8ed8-ee230ca97e9a',
 13: '6043829c-dc8d-4559-840d-435b85c7702c',
 14: '30616b7d-1acf-4541-a8a0-36a9cefe2f85',
 15: '174e242e-9de0-42bc-9410-7e440008b584',
 16: '830edb1c-32ed-4b93-9674-2869bf6b8b40',
 17: '4b09cc93-39c0-4f94-ad68-acdf3280d697',
 18: 'ba127393-6a19-4ebb-8e0f-a7fa8163f943',
 19: 'e2b330ae-e286-4985-9563-8ee7f4cbacb2',
 20: '49c2387e-d771-4f44-98c9-1c5b1dbfcb77',
 21: 'b4994380-6c43-42ac-a212-f4b056a5fee5',
 22: '785bf437-902f-

In [86]:
db.docstore._dict

{'7ee53fe1-b0b3-4ce4-80c7-b7c51aab7820': Document(id='7ee53fe1-b0b3-4ce4-80c7-b7c51aab7820', metadata={'page': 1, 'coordinates': [[{'x': 0.2423, 'y': 0.286}, {'x': 0.7444, 'y': 0.286}, {'x': 0.7444, 'y': 0.4262}, {'x': 0.2423, 'y': 0.4262}], [{'x': 0.2496, 'y': 0.469}, {'x': 0.7543, 'y': 0.469}, {'x': 0.7543, 'y': 0.5318}, {'x': 0.2496, 'y': 0.5318}], [{'x': 0.2509, 'y': 0.5614}, {'x': 0.7581, 'y': 0.5614}, {'x': 0.7581, 'y': 0.6465}, {'x': 0.2509, 'y': 0.6465}], [{'x': 0.2504, 'y': 0.6736}, {'x': 0.6996, 'y': 0.6736}, {'x': 0.6996, 'y': 0.7134}, {'x': 0.2504, 'y': 0.7134}]]}, page_content='창조적 사고로\n세상의 문제를 해결하는\n아주 히어로 아주대학교는 산학협력과 기술이전을 통해 사회 문제에 해답을 제시하며,\n학생들이 스스로 문제를 발견하고 해결하는 힘을 기를 수 있도록 돕습니다.\n아주인들은 유연한 사고와 도전정신으로 새로운 난제에 당당히 맞서고 있습니다. 우리는 시대적 위기를 교육혁신의 기회로 전환하고,\n학문 간 경계를 허물어 더 나은 미래를 위한 해법을 모색합니다.\n혁신적인 교육과 연구, 실무 중심의 취·창업 지원, 다양한 현장 실습 프로그램까지.\n이 모든 것이 당신을 세상의 문제를 해결하는 히어로로 성장시키는 발판입니다. 아주대학교는 히어로들이 세상을 변화시키는 비밀기지입니다.\n지금, 이곳에서 새로운 시대의 히어로가 될 당신의 이야기가 시작됩니다.'),
 '1ebde446-3fd

In [91]:
# 검색 테스트
query = "아주대 기계공학과 고교추천전형 몇명 모집해??"
results = db.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n=== {i+1}번째 결과 ===")
    print(f"내용: {doc.page_content[:700]}...")
    print(f"페이지: {doc.metadata.get('page', 'N/A')}")


=== 1번째 결과 ===
내용: 3 전형별, 모집단위별 모집인원 1 정원내 계열 대학 모집단위 수시모집 정시모집
 고교 추천 전형 ACE 전형 첨단 융합 인재 전형 고른 기회1 전형 고른 기회2 전형 논술 우수자 전형 체육 우수자 (축구) 전형 나군 다군
 일반 전형1 일반 전형2 일반 전형3 일반 전형4 (교차)
 자연 인문 공과대학 다산학부 기계공학과 23 자유전공학부(인문) 365 42 0 6 5 45 5 0 0 0 33 15 0
 산업공학과 20 23 0 4 2 0 0 0 0 20 0
 화학공학과 5 17 0 1 1 5 0 0 0 7 0
 첨단신소재공학과 15 0 36 2 2 8 0 0 0 22 0
 응용화학과 6 0 17 1 1 0 0 0 0 146 7 0
 환경안전공학과 5 13 0 2 0 0 0 0 0 11 0
 건설시스템공학과 7 13 0 2 91 1 0 0 0 0 7 0
 교통시스템공학과 7 13 0 약학과 3 0 0 0 0 0 7 0
 건축학과 10 23 0 3 2 11 0 0 0 10 0
 융합시스템공학과 0 0 0 0 0 0 0 0 0 0 0
 첨단ICT 융합대학 전자공학과 30 32 0 11 5 37 합계 0 0 0 32 0
 지능형반도체공학과 5 0 자유전공학부(자연) 10 2 1 5 0 0 0 17 0
 미래모빌리티공학과 22 0 42 7 5 14 0 0 0 47 0 0
 소프트웨어 소프트웨어학과 11 16 30 4 2 10 0 0 0 0...
페이지: 22

=== 2번째 결과 ===
내용: grade S
10,000,000,000 교육부 대학혁신지원사업
교육혁신성과 최고 S등급
(19억원 증액, 총 100억원 확보) over
20,000,000,000 반도체특성화대학 지원사업 (118억원)
첨단분야 혁신융합대학사업(76억원)
BK21사업단 5개 확보,
혁신지원사업비(약 11억원 추가 확보) G-lamp
148+88 교육혁신과 첨단분야 투자,
압도적 성과로 증명 G-램프 사업
(88억원 증액, 총 236억원 확보) 세계가 인정한

In [ ]:
vectorstore = FAISS.from_documents(docs, hf_embddings)

In [70]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv


In [ ]:
load_dotenv()


llm = ChatAnthropic(
    model="claude-3-5-haiku-20241022", temperature=0  # 최신 Claude3.5 Sonnet 예시
)


# 📝 문서 포맷팅 함수 (중요!)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = ChatPromptTemplate.from_template(
    """
아래 문서를 참고하여 질문에 답하세요.
모르겠으면 모른다고 솔직하게 답하세요.

문서:
{context}

질문: {question}
"""
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [94]:
# 질문하기 - 파이프 방식
question = "아주대 교환학생 제도 알려줘 ?"
response = rag_chain.invoke(question)  # 직접 문자열 전달 (딕셔너리가 아님!)
print("💬 AI의 답변:\n")
print(response)  # StrOutputParser() 덕분에 바로 문자열로 출력됨

💬 AI의 답변:

아주대학교의 교환학생 제도는 다음과 같습니다:

1. 규모
- 전 세계 67개국, 345개교, 4개 기관과 교류협정 (2023년 기준)

2. 지역별 해외파견 비율
- 유럽: 75%
- 아메리카: 11%
- 아시아: 14%

3. 주요 특징
- 1인당 최대 3회까지 교환학생 기회 제공
- 매년 평균 200~300명의 재학생 파견
- 우수 학생 중 장학 대상자 선발 시 장학금 수혜 가능

4. 목표
- "진짜 교실은 세계에 있다"는 슬로건 아래 글로벌 감각과 경험을 쌓는 것을 목적으로 함

이러한 교환학생 프로그램을 통해 학생들은 다양한 문화와 교육 환경을 경험할 수 있습니다.
